In [1]:
from google.colab import drive
import os

drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [3]:
data_path = '/content/drive/MyDrive/UIT SUBJECT/Big Data/Data/output.csv'
data = pd.read_csv(data_path)

data

,Index,Comment,Object,Label,tu_viet_tat,meaning,xoa_khong
0,0,garnacho bảnh quá,garnacho,1.0,NaN,NaN,NaN
1,1,bình an đăng lại kìa,NaN,0.0,NaN,NaN,NaN
2,2,gọi garnacho là gì,garnacho,0.0,NaN,NaN,NaN
3,3,"chỉnh sửa đẹp , ngầu âm thanh chất lượng nhưng...",NaN,0.0,k,không,NaN
4,4,hâm mộ manchester city những vẫn thích united,"manchester city, manchester united",1.0,"mc, mu","manchester city, manchester united",NaN
...,...,...,...,...,...,...,...
5109,5573,romelu lukaku khiến tôi bán xe rồi,romelu lukaku,2.0,lukacặk,romelu lukaku,NaN
5110,5574,bỉ là đội yếu chứ không mạnh như mầy tưởng,bỉ,2.0,NaN,NaN,NaN
5111,5575,đồng tiền lái đôi chân đá trượt bóng luôn khi ...,NaN,2.0,NaN,NaN,NaN
5112,5576,vứt lịch thi đấu mới quan trọng còn nhận định ...,NaN,2.0,NaN,NaN,NaN


In [4]:
data = data[['Comment', 'Object', 'Label']]

In [5]:
predict_data = data[~data['Label'].isin([0, 1, 2])].reset_index(drop=True)
data = data[data['Label'].isin([0, 1, 2])].reset_index(drop=True)

In [6]:
data = data[['Comment', 'Object']]
predict_data = predict_data[['Comment', 'Object']]

data

,Comment,Object
0,garnacho bảnh quá,garnacho
1,bình an đăng lại kìa,NaN
2,gọi garnacho là gì,garnacho
3,"chỉnh sửa đẹp , ngầu âm thanh chất lượng nhưng...",NaN
4,hâm mộ manchester city những vẫn thích united,"manchester city, manchester united"
...,...,...
2322,romelu lukaku khiến tôi bán xe rồi,romelu lukaku
2323,bỉ là đội yếu chứ không mạnh như mầy tưởng,bỉ
2324,đồng tiền lái đôi chân đá trượt bóng luôn khi ...,NaN
2325,vứt lịch thi đấu mới quan trọng còn nhận định ...,NaN


# Huấn luyện mô hình

In [7]:
data.isnull().sum()

Comment      0
Object     957
dtype: int64

In [8]:
# Chuẩn bị dữ liệu
data['Object'] = data['Object'].fillna('')
data['Object'] = data['Object'].apply(lambda x: x.split(',') if x else [])

data

,Comment,Object
0,garnacho bảnh quá,[garnacho]
1,bình an đăng lại kìa,[]
2,gọi garnacho là gì,[garnacho]
3,"chỉnh sửa đẹp , ngầu âm thanh chất lượng nhưng...",[]
4,hâm mộ manchester city những vẫn thích united,"[manchester city, manchester united]"
...,...,...
2322,romelu lukaku khiến tôi bán xe rồi,[romelu lukaku]
2323,bỉ là đội yếu chứ không mạnh như mầy tưởng,[bỉ]
2324,đồng tiền lái đôi chân đá trượt bóng luôn khi ...,[]
2325,vứt lịch thi đấu mới quan trọng còn nhận định ...,[]


In [9]:
# Tạo đặc trưng từ comment sử dụng CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Comment'])

# Chuẩn bị nhãn cho các chủ thể
all_objects = list(set([obj.strip() for sublist in data['Object'] for obj in sublist]))
Y = np.zeros((data.shape[0], len(all_objects)))

for i, objects in enumerate(data['Object']):
    for obj in objects:
        if obj.strip() in all_objects:
            Y[i, all_objects.index(obj.strip())] = 1

In [10]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, Y_train, Y_test, train_index, test_index = train_test_split(X, Y, data.index, test_size=0.2, random_state=42)

# Kiểm tra và loại bỏ các nhãn chỉ chứa một lớp
non_single_class_indices = [i for i in range(Y_train.shape[1]) if len(np.unique(Y_train[:, i])) > 1]
Y_train = Y_train[:, non_single_class_indices]
Y_test = Y_test[:, non_single_class_indices]
all_objects = [all_objects[i] for i in non_single_class_indices]

# Huấn luyện mô hình Logistic Regression
lr = LogisticRegression(max_iter=1000)
multi_target_lr = MultiOutputClassifier(lr, n_jobs=-1)
multi_target_lr.fit(X_train, Y_train)

MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000), n_jobs=-1)

Predict- Tập test

In [11]:
# Dự đoán trên dữ liệu kiểm tra
Y_pred = multi_target_lr.predict(X_test)

In [12]:
# Chuyển đổi nhãn từ ma trận nhị phân thành danh sách các chủ thể
def convert_to_entities(y):
    return [', '.join([all_objects[i] for i in range(len(all_objects)) if y[j, i] == 1]) for j in range(y.shape[0])]

# Tạo DataFrame với các comment và dự đoán
test_comments = data['Comment'].iloc[test_index]
test_actual_objects = convert_to_entities(Y_test)
test_predicted_objects = convert_to_entities(Y_pred)

test_results = pd.DataFrame({
    'Comment': test_comments,
    'Actual_Objects': test_actual_objects,
    'Predicted_Objects': test_predicted_objects
})

In [13]:
# Hiển thị kết quả
test_results

,Comment,Actual_Objects,Predicted_Objects
1736,tối qua bồ đào nha nhiều ông đá như chơi không...,bồ đào nha,
755,cristiano ronaldo là màu mè nhất,cristiano ronaldo,cristiano ronaldo
1669,quang hải đi qua nước ngoài nhặt bóng rồi,quang hải,quang hải
1322,yahh,,
1870,quá là xuất sắc việt namm ơii,việt nam,
...,...,...,...
1360,máy chạy top,,
1989,và con tim đã vui trở lại tuyệt vời việt nam ơi,việt nam,việt nam
786,di dimaria,dimaria,dimaria
2280,bỉ 2 1 pháp,"bỉ, pháp",


Thông số kiểm tra

In [14]:
accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='micro')
recall = recall_score(Y_test, Y_pred, average='micro')
f1 = f1_score(Y_test, Y_pred, average='micro')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.7811158798283262
Precision: 0.9158415841584159
Recall: 0.622895622895623
F1 Score: 0.7414829659318638


# Xuất mô hình

In [15]:
import joblib

joblib.dump(multi_target_lr, '/content/drive/MyDrive/UIT SUBJECT/Big Data/Model/Detection/Machine learning/Logistic Regression/multi_target_lr_model.pkl')
joblib.dump(vectorizer, '/content/drive/MyDrive/UIT SUBJECT/Big Data/Model/Detection/Machine learning/Logistic Regression/vectorizer.pkl')
joblib.dump(all_objects, '/content/drive/MyDrive/UIT SUBJECT/Big Data/Model/Detection/Machine learning/Logistic Regression/all_objects.pkl')

['/content/drive/MyDrive/UIT SUBJECT/Big Data/Model/Detection/Machine learning/Logistic Regression/all_objects.pkl']

# Dự đoán các cột còn lại

In [16]:
predict_data = predict_data[['Comment']]

In [17]:
predict_data

,Comment
0,nice edits but try with english songs
1,_tzn05_ coi tới cuối
2,thủ môn mà nén xa thế
3,nhìn suarez đúng trất có cũng không sang nổi
4,những người nghèo khó đều là chân sút vĩ đạn
...,...
2782,đội tuyển anh vô địch rồi chắc chắn luôn
2783,cho hoi ti số 21 tho an may
2784,vào vòng loại trực tiếp xem còn diễn được nữa ...
2785,anh đi 1 lèo đến chung kết là chắc


In [18]:
# Hàm dự đoán các chủ thể trong comment
def predict_entities(comment):
    X_new = vectorizer.transform([comment])
    Y_new_pred = multi_target_lr.predict(X_new)
    entities = [all_objects[i] for i in range(len(all_objects)) if Y_new_pred[0, i] == 1]
    return ', '.join(entities)

In [19]:
comment = 'hihihihi messi'
predict_entities(comment)

'messi'